In [1]:
%load_ext autoreload
%autoreload 2

# Exploring the strengts and weaknesses of Stable Diffusion XL 0.9
This notebook aims at uncovering strengths and weaknesses of the current Stable Diffusion XL 0.9 model.

**Note that it builds on [THIS NOTEBOOK](stable_diffusion_evaluation.ipynb) which generates the necessary data. Run this in order to follow along or just use your own data.**

# Step 1: Loading the data

In [2]:
# Imports
import pandas as pd
import numpy as np
from sliceguard import SliceGuard
from renumics.spotlight import Image, Embedding

In [3]:
# Load the dataset
df = pd.read_json("sd_dataset_scored_embedded_parti.json")

In [4]:
# Get the text and image embeddings from the dataframe
clip_text_embeddings = np.vstack(df["clip_text_embedding"])
clip_image_embeddings = np.vstack(df["clip_image_embedding"])

# Step 2: Category-based analysis

In [5]:
# Define a metric that simply returns the precomputed metric
def return_precomputed_metric(y, y_pred):
    return y.mean(0)

In [6]:
sg = SliceGuard()

# Show the drop and support levels that could make sense
sg.show(df, ["category"],
               "clip_score",
               "clip_score",
               return_precomputed_metric)

Please wait. sliceguard is preparing your data.
Feature category was inferred as being categorical. Will be treated as nominal by default. If ordinal specify in feature_types and feature_orders!


In [7]:
# Find category specific issues
issues = sg.find_issues(df, ["category"],
               "clip_score",
               "clip_score",
               return_precomputed_metric,
               min_support=50,
               min_drop=0.5)
sg.report(spotlight_dtype={"image": Image})

Feature category was inferred as being categorical. Will be treated as nominal by default. If ordinal specify in feature_types and feature_orders!
The overall metric value is 28.02405169257253
Using 50 as minimum support for determining problematic clusters.
Using 0.5 as minimum drop for determining problematic clusters.
Identified 6 problematic slices.


,image,prompt,clip_score,clip_text_embedding,clip_image_embedding,category,challenge
0,images_parti/0e37fbf7-73f5-4a78-afdc-5b330960f...,bond,22.219160,"[0.0051389872, 0.0128664747, -5.1433e-06, -0.0...","[0.0517092124, -0.0056039845, -0.0363197513, -...",Abstract,Basic
1,images_parti/5dfe64c4-1306-4653-9844-07c81e92a...,element,17.738974,"[-0.016821011900000002, -0.0035089231, 0.01665...","[0.0090920646, 0.0442848727, -0.0103910565, -0...",Abstract,Basic
2,images_parti/2cab0a4d-9655-4bcc-8ec0-0674fa590...,molecule,22.847107,"[0.0322200023, 0.0193178039, 0.0355037935, 0.0...","[0.06821470710000001, 0.021988526, 0.005686196...",Abstract,Basic
3,images_parti/298b06fb-3124-43a6-9c21-3e389d1c3...,life,13.133687,"[-0.0176262874, 0.0031878012, 0.0181580558, -0...","[0.0233738441, 0.0302052088, 0.0203849468, 0.0...",Abstract,Basic
4,images_parti/00faabcb-3e33-416a-b6a6-ff7079277...,protein,27.205805,"[-0.0027263297000000002, -0.0371992737, 0.0142...","[0.0372581743, 0.0713513345, -0.0159365907, 0....",Abstract,Basic
...,...,...,...,...,...,...,...
1626,images_parti/095e9c31-b2de-4348-9fc0-fd0924960...,a wooden post with a yellow '3' painted on top,26.527929,"[-0.0244453568, 0.0038155904000000004, 0.02567...","[0.0230240077, 0.0911815539, 0.0024713683, -0....",Outdoor Scenes,Writing & Symbols
1627,images_parti/d7536a7b-d0b3-4aee-a5d5-b0b08486e...,a wooden post in front of a patch of tall grass,35.294491,"[-0.0139579102, 0.0357775502, 0.0248423312, -0...","[-0.0208352543, 0.056416120400000005, -0.00187...",Outdoor Scenes,Writing & Symbols
1628,images_parti/b479a636-f06f-4089-a291-e5fef15aa...,a wooden post with a blue '5' painted on top,23.615295,"[-0.0122707775, -0.0098742479, -0.0016869248, ...","[-0.019069608300000002, 0.0696209222, 0.002791...",Outdoor Scenes,Writing & Symbols
1629,images_parti/b14a1a01-7ae6-45f3-a559-96159467c...,a series of musical notes on a black t-shirt,30.207697,"[0.006846202500000001, 0.0254821796, 0.0078348...","[0.0231587067, -0.0018685912, 0.00455781070000...",Artifacts,Writing & Symbols


# Step 3: Challenge-based analysis

In [8]:
sg = SliceGuard()

# Show the drop and support levels that could make sense
sg.show(df, ["challenge"],
               "clip_score",
               "clip_score",
               return_precomputed_metric)

Please wait. sliceguard is preparing your data.
Feature challenge was inferred as being categorical. Will be treated as nominal by default. If ordinal specify in feature_types and feature_orders!


In [9]:
# Find challenge specific issues
sg = SliceGuard()

# Show the drop and support levels that could make sense
# for the category feature
sg.find_issues(df, ["challenge"],
            "clip_score",
            "clip_score",
            return_precomputed_metric,
            min_drop=1,
            min_support=20)
sg.report(spotlight_dtype={"image": Image})

Feature challenge was inferred as being categorical. Will be treated as nominal by default. If ordinal specify in feature_types and feature_orders!
The overall metric value is 28.02405169257253
Using 20 as minimum support for determining problematic clusters.
Using 1 as minimum drop for determining problematic clusters.
Identified 5 problematic slices.


,image,prompt,clip_score,clip_text_embedding,clip_image_embedding,category,challenge
0,images_parti/0e37fbf7-73f5-4a78-afdc-5b330960f...,bond,22.219160,"[0.0051389872, 0.0128664747, -5.1433e-06, -0.0...","[0.0517092124, -0.0056039845, -0.0363197513, -...",Abstract,Basic
1,images_parti/5dfe64c4-1306-4653-9844-07c81e92a...,element,17.738974,"[-0.016821011900000002, -0.0035089231, 0.01665...","[0.0090920646, 0.0442848727, -0.0103910565, -0...",Abstract,Basic
2,images_parti/2cab0a4d-9655-4bcc-8ec0-0674fa590...,molecule,22.847107,"[0.0322200023, 0.0193178039, 0.0355037935, 0.0...","[0.06821470710000001, 0.021988526, 0.005686196...",Abstract,Basic
3,images_parti/298b06fb-3124-43a6-9c21-3e389d1c3...,life,13.133687,"[-0.0176262874, 0.0031878012, 0.0181580558, -0...","[0.0233738441, 0.0302052088, 0.0203849468, 0.0...",Abstract,Basic
4,images_parti/00faabcb-3e33-416a-b6a6-ff7079277...,protein,27.205805,"[-0.0027263297000000002, -0.0371992737, 0.0142...","[0.0372581743, 0.0713513345, -0.0159365907, 0....",Abstract,Basic
...,...,...,...,...,...,...,...
1626,images_parti/095e9c31-b2de-4348-9fc0-fd0924960...,a wooden post with a yellow '3' painted on top,26.527929,"[-0.0244453568, 0.0038155904000000004, 0.02567...","[0.0230240077, 0.0911815539, 0.0024713683, -0....",Outdoor Scenes,Writing & Symbols
1627,images_parti/d7536a7b-d0b3-4aee-a5d5-b0b08486e...,a wooden post in front of a patch of tall grass,35.294491,"[-0.0139579102, 0.0357775502, 0.0248423312, -0...","[-0.0208352543, 0.056416120400000005, -0.00187...",Outdoor Scenes,Writing & Symbols
1628,images_parti/b479a636-f06f-4089-a291-e5fef15aa...,a wooden post with a blue '5' painted on top,23.615295,"[-0.0122707775, -0.0098742479, -0.0016869248, ...","[-0.019069608300000002, 0.0696209222, 0.002791...",Outdoor Scenes,Writing & Symbols
1629,images_parti/b14a1a01-7ae6-45f3-a559-96159467c...,a series of musical notes on a black t-shirt,30.207697,"[0.006846202500000001, 0.0254821796, 0.0078348...","[0.0231587067, -0.0018685912, 0.00455781070000...",Artifacts,Writing & Symbols


# Step 4: Challenge, Category interaction

In [ ]:
sg = SliceGuard()

# Show the drop and support levels that could make sense
sg.show(df, ["challenge", "category"],
               "clip_score",
               "clip_score",
               return_precomputed_metric)

In [ ]:
# Find issues based on combinations of category and challenge.
sg = SliceGuard()

sg.find_issues(df, ["challenge", "category"],
            "clip_score",
            "clip_score",
            return_precomputed_metric,
            min_drop=1,
            min_support=20)
sg.report(spotlight_dtype={"image": Image})

# Step 5: Analysis based on prompt embeddings

In [10]:
sg = SliceGuard()

# Show the drop and support levels that could make sense
sg.show(df, ["clip_text_embedding"],
       "clip_score",
       "clip_score",
       return_precomputed_metric,
        precomputed_embeddings={"clip_text_embedding": clip_text_embeddings})

Please wait. sliceguard is preparing your data.


In [17]:
sg = SliceGuard()

# Show the drop and support levels that could make sense
issues = sg.find_issues(df, ["clip_text_embedding"],
       "clip_score",
       "clip_score",
       return_precomputed_metric,
        min_support=3,
        min_drop=6,
        precomputed_embeddings={"clip_text_embedding": clip_text_embeddings})
sg.report(spotlight_dtype={"image": Image})

The overall metric value is 28.02405169257253
Using 3 as minimum support for determining problematic clusters.
Using 6 as minimum drop for determining problematic clusters.
Identified 13 problematic slices.


,image,prompt,clip_score,clip_text_embedding,clip_image_embedding,category,challenge,sg_emb_clip_text_embedding
0,images_parti/0e37fbf7-73f5-4a78-afdc-5b330960f...,bond,22.219160,"[0.0051389872, 0.0128664747, -5.1433e-06, -0.0...","[0.0517092124, -0.0056039845, -0.0363197513, -...",Abstract,Basic,"[0.0051389872, 0.0128664747, -5.1433e-06, -0.0..."
1,images_parti/5dfe64c4-1306-4653-9844-07c81e92a...,element,17.738974,"[-0.016821011900000002, -0.0035089231, 0.01665...","[0.0090920646, 0.0442848727, -0.0103910565, -0...",Abstract,Basic,"[-0.016821011900000002, -0.0035089231, 0.01665..."
2,images_parti/2cab0a4d-9655-4bcc-8ec0-0674fa590...,molecule,22.847107,"[0.0322200023, 0.0193178039, 0.0355037935, 0.0...","[0.06821470710000001, 0.021988526, 0.005686196...",Abstract,Basic,"[0.0322200023, 0.0193178039, 0.0355037935, 0.0..."
3,images_parti/298b06fb-3124-43a6-9c21-3e389d1c3...,life,13.133687,"[-0.0176262874, 0.0031878012, 0.0181580558, -0...","[0.0233738441, 0.0302052088, 0.0203849468, 0.0...",Abstract,Basic,"[-0.0176262874, 0.0031878012, 0.0181580558, -0..."
4,images_parti/00faabcb-3e33-416a-b6a6-ff7079277...,protein,27.205805,"[-0.0027263297000000002, -0.0371992737, 0.0142...","[0.0372581743, 0.0713513345, -0.0159365907, 0....",Abstract,Basic,"[-0.0027263297000000002, -0.0371992737, 0.0142..."
...,...,...,...,...,...,...,...,...
1626,images_parti/095e9c31-b2de-4348-9fc0-fd0924960...,a wooden post with a yellow '3' painted on top,26.527929,"[-0.0244453568, 0.0038155904000000004, 0.02567...","[0.0230240077, 0.0911815539, 0.0024713683, -0....",Outdoor Scenes,Writing & Symbols,"[-0.0244453568, 0.0038155904000000004, 0.02567..."
1627,images_parti/d7536a7b-d0b3-4aee-a5d5-b0b08486e...,a wooden post in front of a patch of tall grass,35.294491,"[-0.0139579102, 0.0357775502, 0.0248423312, -0...","[-0.0208352543, 0.056416120400000005, -0.00187...",Outdoor Scenes,Writing & Symbols,"[-0.0139579102, 0.0357775502, 0.0248423312, -0..."
1628,images_parti/b479a636-f06f-4089-a291-e5fef15aa...,a wooden post with a blue '5' painted on top,23.615295,"[-0.0122707775, -0.0098742479, -0.0016869248, ...","[-0.019069608300000002, 0.0696209222, 0.002791...",Outdoor Scenes,Writing & Symbols,"[-0.0122707775, -0.0098742479, -0.0016869248, ..."
1629,images_parti/b14a1a01-7ae6-45f3-a559-96159467c...,a series of musical notes on a black t-shirt,30.207697,"[0.006846202500000001, 0.0254821796, 0.0078348...","[0.0231587067, -0.0018685912, 0.00455781070000...",Artifacts,Writing & Symbols,"[0.006846202500000001, 0.0254821796, 0.0078348..."
